# Session 10: Hybrid Architecture Design I
## LLM + Symbolic Reasoning

**Production LLM Deployment: Risk Characterization Before Failure**

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Javihaus/Production_LLM_Deployment/blob/main/sessions/session_10_hybrid_architecture_i/notebook.ipynb)

---

**Learning Objectives:**
1. Decide when hybrid architectures are necessary
2. Design LLM + symbolic reasoning systems
3. Implement temporal constraint checkers
4. Specify component responsibilities clearly

## Setup

In [ ]:
!pip install -q anthropic numpy pandas matplotlib seaborn

import anthropic
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from dataclasses import dataclass
from typing import List, Dict, Tuple, Optional, Any
from enum import Enum
from abc import ABC, abstractmethod
import json
import time

plt.style.use('seaborn-v0_8-whitegrid')
%matplotlib inline

try:
    from google.colab import userdata
    api_key = userdata.get('ANTHROPIC_API_KEY')
except:
    import os
    api_key = os.environ.get('ANTHROPIC_API_KEY')

client = anthropic.Anthropic(api_key=api_key)
print("Setup complete!")

## Part 1: When Are Hybrid Architectures Necessary?

Based on our failure mode analysis, hybrid architectures are required when:

| Requirement | Why LLM-Only Fails | Hybrid Solution |
|-------------|-------------------|------------------|
| Deterministic outputs | LLMs are probabilistic | Add verification module |
| Temporal reasoning | Discrete tokens fail | Add temporal checker |
| Constraint satisfaction | Approximation only | Add constraint solver |
| Verifiable reasoning | Black box | Add symbolic reasoner |
| Factual accuracy | Hallucination risk | Add retrieval (RAG) |

In [ ]:
class HybridDecisionFramework:
    """Framework for deciding when hybrid architectures are needed."""
    
    DECISION_CRITERIA = {
        "temporal_reasoning": {
            "question": "Does your application require temporal constraint checking?",
            "examples": ["medication timing", "scheduling", "deadlines"],
            "llm_reliability": "VERY LOW",
            "hybrid_component": "Temporal Constraint Checker"
        },
        "deterministic_output": {
            "question": "Must outputs be deterministic and reproducible?",
            "examples": ["compliance checking", "safety decisions", "financial calculations"],
            "llm_reliability": "LOW",
            "hybrid_component": "Verification Module"
        },
        "constraint_satisfaction": {
            "question": "Are there hard constraints that must always be satisfied?",
            "examples": ["resource allocation", "scheduling conflicts", "rule compliance"],
            "llm_reliability": "LOW",
            "hybrid_component": "Constraint Solver"
        },
        "factual_accuracy": {
            "question": "Is factual accuracy critical with low tolerance for hallucination?",
            "examples": ["medical information", "legal citations", "technical specifications"],
            "llm_reliability": "MEDIUM",
            "hybrid_component": "Retrieval-Augmented Generation (RAG)"
        },
        "audit_requirements": {
            "question": "Must reasoning steps be traceable and auditable?",
            "examples": ["regulatory compliance", "legal decisions", "financial audits"],
            "llm_reliability": "LOW",
            "hybrid_component": "Symbolic Reasoner with Logging"
        }
    }
    
    def assess(self, requirements: List[str]) -> Dict:
        """Assess whether hybrid architecture is needed."""
        applicable = []
        components_needed = []
        
        for req in requirements:
            if req in self.DECISION_CRITERIA:
                criteria = self.DECISION_CRITERIA[req]
                applicable.append({
                    "requirement": req,
                    "llm_reliability": criteria["llm_reliability"],
                    "component": criteria["hybrid_component"]
                })
                components_needed.append(criteria["hybrid_component"])
        
        hybrid_needed = any(
            c["llm_reliability"] in ["VERY LOW", "LOW"] 
            for c in applicable
        )
        
        return {
            "hybrid_needed": hybrid_needed,
            "applicable_criteria": applicable,
            "components_needed": list(set(components_needed)),
            "recommendation": self._get_recommendation(hybrid_needed, applicable)
        }
    
    def _get_recommendation(self, hybrid_needed: bool, applicable: List) -> str:
        if not hybrid_needed:
            return "Pure LLM approach may be sufficient. Proceed with careful testing."
        
        low_reliability = [a for a in applicable if a["llm_reliability"] in ["VERY LOW", "LOW"]]
        components = [a["component"] for a in low_reliability]
        
        return f"Hybrid architecture required. Add: {', '.join(set(components))}"


# Example: Medical triage system
framework = HybridDecisionFramework()

assessment = framework.assess([
    "temporal_reasoning",
    "deterministic_output",
    "factual_accuracy"
])

print("HYBRID ARCHITECTURE ASSESSMENT")
print("=" * 50)
print(f"Hybrid Needed: {assessment['hybrid_needed']}")
print(f"\nComponents Needed:")
for comp in assessment['components_needed']:
    print(f"  - {comp}")
print(f"\nRecommendation: {assessment['recommendation']}")

## Part 2: Architecture Pattern 1 - LLM + Temporal Constraint Checker

The most common hybrid pattern for medical and scheduling applications.

In [ ]:
# Base classes for hybrid components

class SymbolicChecker(ABC):
    """Abstract base class for symbolic checking modules."""
    
    @abstractmethod
    def validate(self, extracted_data: Dict) -> Dict:
        """Validate extracted data and return result."""
        pass


@dataclass
class ValidationResult:
    """Result of symbolic validation."""
    is_valid: bool
    explanation: str
    violations: List[str]
    confidence: float = 1.0  # Symbolic = always 1.0


class TemporalConstraintChecker(SymbolicChecker):
    """Deterministic temporal constraint validation."""
    
    def __init__(self):
        self.constraints = []
    
    def add_constraint(self, name: str, min_hours: float):
        """Add a minimum time gap constraint."""
        self.constraints.append({"name": name, "min_hours": min_hours})
    
    def _time_to_minutes(self, time_str: str) -> float:
        """Convert time string to minutes from midnight."""
        time_str = time_str.upper().strip()
        is_pm = 'PM' in time_str
        is_am = 'AM' in time_str
        time_str = time_str.replace('AM', '').replace('PM', '').strip()
        
        parts = time_str.replace(':', ' ').split()
        hours = int(parts[0])
        minutes = int(parts[1]) if len(parts) > 1 else 0
        
        if is_pm and hours != 12:
            hours += 12
        elif is_am and hours == 12:
            hours = 0
        
        return hours * 60 + minutes
    
    def validate(self, extracted_data: Dict) -> ValidationResult:
        """Validate temporal constraints."""
        violations = []
        
        event_a_time = self._time_to_minutes(extracted_data.get('event_a_time', '0:00'))
        event_b_time = self._time_to_minutes(extracted_data.get('event_b_time', '0:00'))
        min_gap_hours = extracted_data.get('min_gap_hours', 0)
        
        elapsed_minutes = event_b_time - event_a_time
        required_minutes = min_gap_hours * 60
        
        is_valid = elapsed_minutes >= required_minutes
        
        if not is_valid:
            violations.append(
                f"Insufficient time gap: {elapsed_minutes/60:.2f}h elapsed, "
                f"{min_gap_hours}h required"
            )
        
        explanation = (
            f"Time elapsed: {elapsed_minutes/60:.2f} hours. "
            f"Required: {min_gap_hours} hours. "
            f"{'VALID' if is_valid else 'INVALID'}"
        )
        
        return ValidationResult(
            is_valid=is_valid,
            explanation=explanation,
            violations=violations
        )


# Test the checker
checker = TemporalConstraintChecker()

test_data = {
    'event_a_time': '8:00 AM',
    'event_b_time': '11:00 AM',
    'min_gap_hours': 4
}

result = checker.validate(test_data)
print(f"Valid: {result.is_valid}")
print(f"Explanation: {result.explanation}")
print(f"Violations: {result.violations}")

In [ ]:
class LLMExtractor:
    """LLM component for extracting structured information from natural language."""
    
    def __init__(self, client):
        self.client = client
    
    def extract_temporal_info(self, scenario: str) -> Optional[Dict]:
        """Extract temporal information from natural language."""
        
        prompt = f"""Extract the temporal information from this scenario.
Return ONLY a JSON object with these exact fields:
- event_a_time: time of first event (format: "HH:MM AM/PM")
- event_b_time: time of second event (format: "HH:MM AM/PM")
- min_gap_hours: minimum required gap in hours (number)

Scenario: {scenario}

JSON:"""
        
        response = self.client.messages.create(
            model="claude-sonnet-4-5-20250929",
            max_tokens=200,
            messages=[{"role": "user", "content": prompt}]
        )
        
        text = response.content[0].text
        
        # Parse JSON from response
        start = text.find('{')
        end = text.rfind('}') + 1
        if start >= 0 and end > start:
            try:
                return json.loads(text[start:end])
            except json.JSONDecodeError:
                return None
        return None


# Test extraction
extractor = LLMExtractor(client)

scenario = "I took aspirin at 8:00 AM. The doctor said to wait at least 4 hours before taking ibuprofen. It's now 11:00 AM - can I take it?"

extracted = extractor.extract_temporal_info(scenario)
print("Extracted information:")
print(json.dumps(extracted, indent=2))

In [ ]:
class HybridTemporalReasoner:
    """Complete hybrid system combining LLM extraction with symbolic verification."""
    
    def __init__(self, client):
        self.extractor = LLMExtractor(client)
        self.checker = TemporalConstraintChecker()
    
    def reason(self, natural_language_input: str) -> Dict:
        """Full hybrid reasoning pipeline."""
        
        # Step 1: LLM extracts structured information
        extracted = self.extractor.extract_temporal_info(natural_language_input)
        
        if not extracted:
            return {
                "success": False,
                "error": "Could not extract temporal information from input",
                "input": natural_language_input
            }
        
        # Step 2: Symbolic checker validates constraints
        validation = self.checker.validate(extracted)
        
        # Step 3: Return combined result
        return {
            "success": True,
            "input": natural_language_input,
            "extracted": extracted,
            "is_valid": validation.is_valid,
            "answer": "YES" if validation.is_valid else "NO",
            "explanation": validation.explanation,
            "violations": validation.violations,
            "confidence": validation.confidence
        }


# Test the complete hybrid system
hybrid = HybridTemporalReasoner(client)

test_scenarios = [
    "I took my blood pressure medication at 7:30 AM. The pharmacist said to wait at least 6 hours before taking the antihistamine. It's now 1:00 PM - is it safe?",
    "Patient received Drug A at 9:00 AM. Protocol requires 4-hour separation before Drug B. Current time: 12:45 PM. Should we administer Drug B?",
    "The morning dose was given at 6:00 AM. Evening dose requires minimum 8 hours gap. Can we give it at 3:00 PM?"
]

print("=" * 60)
print("HYBRID TEMPORAL REASONER - TEST RESULTS")
print("=" * 60)

for scenario in test_scenarios:
    print(f"\nScenario: {scenario[:60]}...")
    result = hybrid.reason(scenario)
    
    if result["success"]:
        print(f"Extracted: {result['extracted']}")
        print(f"Answer: {result['answer']}")
        print(f"Explanation: {result['explanation']}")
    else:
        print(f"Error: {result['error']}")
    print("-" * 40)

## Part 3: Architecture Pattern 2 - LLM + Verification Module

For applications requiring post-hoc verification of LLM outputs.

In [ ]:
class ComplianceVerifier(SymbolicChecker):
    """Verifies LLM outputs against compliance rules."""
    
    def __init__(self):
        self.rules = []
    
    def add_rule(self, name: str, check_fn, error_msg: str):
        """Add a compliance rule."""
        self.rules.append({
            "name": name,
            "check": check_fn,
            "error_msg": error_msg
        })
    
    def validate(self, extracted_data: Dict) -> ValidationResult:
        """Validate against all compliance rules."""
        violations = []
        
        for rule in self.rules:
            if not rule["check"](extracted_data):
                violations.append(f"{rule['name']}: {rule['error_msg']}")
        
        is_valid = len(violations) == 0
        
        return ValidationResult(
            is_valid=is_valid,
            explanation=f"Passed {len(self.rules) - len(violations)}/{len(self.rules)} rules",
            violations=violations
        )


class HybridComplianceSystem:
    """Hybrid system for compliance checking."""
    
    def __init__(self, client):
        self.client = client
        self.verifier = ComplianceVerifier()
        self._setup_rules()
    
    def _setup_rules(self):
        """Set up compliance rules."""
        # Rule 1: Amount must be positive
        self.verifier.add_rule(
            "positive_amount",
            lambda d: d.get('amount', 0) > 0,
            "Amount must be positive"
        )
        
        # Rule 2: Amount must be within limit
        self.verifier.add_rule(
            "amount_limit",
            lambda d: d.get('amount', 0) <= 10000,
            "Amount exceeds $10,000 limit"
        )
        
        # Rule 3: Must have approval for large amounts
        self.verifier.add_rule(
            "approval_required",
            lambda d: d.get('amount', 0) <= 5000 or d.get('has_approval', False),
            "Amounts over $5,000 require approval"
        )
    
    def extract_transaction_info(self, text: str) -> Optional[Dict]:
        """Use LLM to extract transaction information."""
        prompt = f"""Extract the transaction information from this text.
Return ONLY a JSON object with:
- amount: the dollar amount (number)
- has_approval: whether approval was mentioned (boolean)
- description: brief description of transaction

Text: {text}

JSON:"""
        
        response = self.client.messages.create(
            model="claude-sonnet-4-5-20250929",
            max_tokens=200,
            messages=[{"role": "user", "content": prompt}]
        )
        
        text_response = response.content[0].text
        start = text_response.find('{')
        end = text_response.rfind('}') + 1
        
        if start >= 0 and end > start:
            try:
                return json.loads(text_response[start:end])
            except json.JSONDecodeError:
                return None
        return None
    
    def process(self, transaction_text: str) -> Dict:
        """Process transaction through hybrid pipeline."""
        # Step 1: LLM extracts information
        extracted = self.extract_transaction_info(transaction_text)
        
        if not extracted:
            return {"success": False, "error": "Could not extract transaction info"}
        
        # Step 2: Verify against rules
        validation = self.verifier.validate(extracted)
        
        return {
            "success": True,
            "extracted": extracted,
            "compliant": validation.is_valid,
            "explanation": validation.explanation,
            "violations": validation.violations
        }


# Test the compliance system
compliance_system = HybridComplianceSystem(client)

test_transactions = [
    "Process a payment of $3,500 for office supplies.",
    "Transfer $7,500 to vendor. Manager approval obtained.",
    "Reimbursement request for $8,000 conference expenses. No approval yet."
]

print("=" * 60)
print("COMPLIANCE VERIFICATION RESULTS")
print("=" * 60)

for transaction in test_transactions:
    print(f"\nTransaction: {transaction}")
    result = compliance_system.process(transaction)
    
    if result["success"]:
        print(f"Extracted: {result['extracted']}")
        print(f"Compliant: {result['compliant']}")
        if result['violations']:
            print(f"Violations: {result['violations']}")
    print("-" * 40)

## Part 4: Architecture Pattern 3 - LLM + Retrieval (RAG)

For applications requiring factual grounding.

In [ ]:
class SimpleVectorStore:
    """Simple in-memory vector store for demonstration."""
    
    def __init__(self):
        self.documents = []
    
    def add_document(self, doc_id: str, content: str, metadata: Dict = None):
        """Add a document to the store."""
        self.documents.append({
            "id": doc_id,
            "content": content,
            "metadata": metadata or {}
        })
    
    def search(self, query: str, top_k: int = 3) -> List[Dict]:
        """Simple keyword-based search (placeholder for vector search)."""
        query_words = set(query.lower().split())
        
        scored = []
        for doc in self.documents:
            doc_words = set(doc["content"].lower().split())
            overlap = len(query_words & doc_words)
            scored.append((overlap, doc))
        
        scored.sort(key=lambda x: x[0], reverse=True)
        return [doc for _, doc in scored[:top_k]]


class HybridRAGSystem:
    """Hybrid RAG system with retrieval grounding."""
    
    def __init__(self, client):
        self.client = client
        self.vector_store = SimpleVectorStore()
        self._load_knowledge_base()
    
    def _load_knowledge_base(self):
        """Load sample knowledge base."""
        docs = [
            ("med_001", "Aspirin should not be taken with ibuprofen within 4 hours. Both are NSAIDs and can increase bleeding risk."),
            ("med_002", "Acetaminophen (Tylenol) can be taken with NSAIDs as they work differently."),
            ("med_003", "Blood pressure medications should be taken at the same time each day for consistent effect."),
            ("med_004", "Antihistamines may cause drowsiness. Avoid driving until you know how they affect you."),
            ("med_005", "Grapefruit juice can interact with many medications. Consult your pharmacist.")
        ]
        
        for doc_id, content in docs:
            self.vector_store.add_document(doc_id, content)
    
    def answer(self, question: str) -> Dict:
        """Answer question using RAG."""
        # Step 1: Retrieve relevant documents
        retrieved = self.vector_store.search(question, top_k=2)
        
        context = "\n".join([f"- {doc['content']}" for doc in retrieved])
        
        # Step 2: Generate answer with context
        prompt = f"""Answer the question based ONLY on the provided context.
If the context doesn't contain enough information, say "I don't have enough information."

Context:
{context}

Question: {question}

Answer:"""
        
        response = self.client.messages.create(
            model="claude-sonnet-4-5-20250929",
            max_tokens=300,
            messages=[{"role": "user", "content": prompt}]
        )
        
        return {
            "question": question,
            "answer": response.content[0].text,
            "sources": [doc["id"] for doc in retrieved],
            "context_used": context
        }


# Test RAG system
rag_system = HybridRAGSystem(client)

questions = [
    "Can I take aspirin and ibuprofen together?",
    "Is it safe to take Tylenol with other pain medications?",
    "What should I know about antihistamines?"
]

print("=" * 60)
print("RAG SYSTEM - GROUNDED ANSWERS")
print("=" * 60)

for question in questions:
    print(f"\nQ: {question}")
    result = rag_system.answer(question)
    print(f"A: {result['answer'][:200]}...")
    print(f"Sources: {result['sources']}")
    print("-" * 40)

## Part 5: Component Responsibility Matrix

Clear separation of concerns is critical for hybrid architectures.

In [ ]:
# Create responsibility matrix visualization

responsibilities = {
    "Task": [
        "Natural language understanding",
        "Entity extraction",
        "Temporal math",
        "Constraint propagation",
        "Deterministic validation",
        "Human-readable explanations",
        "Ambiguity resolution",
        "Consistency checking",
        "Audit logging"
    ],
    "LLM": [1, 1, 0, 0, 0, 1, 1, 0, 0],
    "Symbolic": [0, 0, 1, 1, 1, 0, 0, 1, 1]
}

df_resp = pd.DataFrame(responsibilities)
df_resp = df_resp.set_index("Task")

fig, ax = plt.subplots(figsize=(10, 8))
sns.heatmap(
    df_resp, 
    annot=True, 
    cmap=["#FFCCCC", "#CCFFCC"],
    cbar=False,
    linewidths=1,
    ax=ax
)
ax.set_title("Component Responsibility Matrix\n(1 = Responsible, 0 = Not Responsible)", fontsize=14)
plt.tight_layout()
plt.show()

## Part 6: Exercise - Design Your Hybrid Architecture

Apply the patterns to your deployment scenario.

In [ ]:
# EXERCISE: Fill in your hybrid architecture specification

class HybridArchitectureSpec:
    """Template for specifying hybrid architecture."""
    
    def __init__(self):
        self.name = "YOUR APPLICATION NAME"
        self.llm_responsibilities = []
        self.symbolic_responsibilities = []
        self.data_flow = []
        self.error_handling = []
    
    def add_llm_responsibility(self, task: str):
        self.llm_responsibilities.append(task)
    
    def add_symbolic_responsibility(self, task: str):
        self.symbolic_responsibilities.append(task)
    
    def add_data_flow_step(self, step: str):
        self.data_flow.append(step)
    
    def add_error_handler(self, error_type: str, handler: str):
        self.error_handling.append({"error": error_type, "handler": handler})
    
    def print_spec(self):
        print("=" * 60)
        print(f"HYBRID ARCHITECTURE: {self.name}")
        print("=" * 60)
        
        print("\nLLM Responsibilities:")
        for r in self.llm_responsibilities:
            print(f"  - {r}")
        
        print("\nSymbolic Component Responsibilities:")
        for r in self.symbolic_responsibilities:
            print(f"  - {r}")
        
        print("\nData Flow:")
        for i, step in enumerate(self.data_flow, 1):
            print(f"  {i}. {step}")
        
        print("\nError Handling:")
        for eh in self.error_handling:
            print(f"  - {eh['error']}: {eh['handler']}")


# Example specification
spec = HybridArchitectureSpec()
spec.name = "Medical Appointment Scheduler"

spec.add_llm_responsibility("Parse natural language appointment requests")
spec.add_llm_responsibility("Extract patient preferences")
spec.add_llm_responsibility("Generate human-friendly confirmation messages")

spec.add_symbolic_responsibility("Check schedule conflicts")
spec.add_symbolic_responsibility("Enforce appointment duration rules")
spec.add_symbolic_responsibility("Validate buffer times between appointments")

spec.add_data_flow_step("User request -> LLM (extract appointment details)")
spec.add_data_flow_step("Extracted details -> Constraint Checker (validate schedule)")
spec.add_data_flow_step("If valid: Confirm booking")
spec.add_data_flow_step("If invalid: Return alternatives")

spec.add_error_handler("Extraction failure", "Ask user for clarification")
spec.add_error_handler("Schedule conflict", "Suggest nearest available times")

spec.print_spec()

## Key Takeaways

1. **Know when hybrid is necessary** - Use the decision framework to assess your requirements

2. **Clear responsibility separation** - LLM handles language, symbolic handles logic

3. **Three main patterns:**
   - LLM + Temporal Checker (scheduling, timing)
   - LLM + Verifier (compliance, safety)
   - LLM + Retrieval (factual grounding)

4. **Design for failure** - Plan error handling at each interface

5. **Test the whole system** - Integration testing is critical

---

**Homework:** Design a complete hybrid architecture specification for your deployment scenario.

**Next Session:** Hybrid Architecture Design II - Integration Patterns